In [15]:
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langchain.messages import HumanMessage,AIMessage
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

In [3]:
@tool
def multiply(a :int, b:int)->int:
    """Given two numbers a and b this tool returns their multiplication
    """
    return a*b


In [4]:
llm = ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")

In [5]:
llm_with_tools = llm.bind_tools([multiply])

In [23]:
query = HumanMessage("can you multiply 3 and 4")

messages = [query]
print(messages)

[HumanMessage(content='can you multiply 3 and 4', additional_kwargs={}, response_metadata={})]


In [25]:
result = (llm_with_tools.invoke("can you multiply 3 and 4"))
messages.append(result)

In [26]:
print(messages)

[HumanMessage(content='can you multiply 3 and 4', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '62gqwrwxq', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 236, 'total_tokens': 255, 'completion_time': 0.037921216, 'completion_tokens_details': None, 'prompt_time': 0.022782843, 'prompt_tokens_details': None, 'queue_time': 0.053850426, 'total_time': 0.060704059}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd222-aa56-7b92-ab69-697a9ab88bbf-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 4}, 'id': '62gqwrwxq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 236, 'output_tokens': 19, 'total_tokens': 255})]


In [27]:
tool_Result = multiply.invoke(result.tool_calls[0])
messages.append(tool_Result)

In [28]:
print(messages)

[HumanMessage(content='can you multiply 3 and 4', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '62gqwrwxq', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 236, 'total_tokens': 255, 'completion_time': 0.037921216, 'completion_tokens_details': None, 'prompt_time': 0.022782843, 'prompt_tokens_details': None, 'queue_time': 0.053850426, 'total_time': 0.060704059}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd222-aa56-7b92-ab69-697a9ab88bbf-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 4}, 'id': '62gqwrwxq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 236, 'output_tokens': 19, 'total_tokens': 255}), ToolMessage(content='12', name='multiply'

In [29]:
llm_with_tools.invoke(messages)

AIMessage(content='The result of the multiplication is 12.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 266, 'total_tokens': 276, 'completion_time': 0.019062332, 'completion_tokens_details': None, 'prompt_time': 0.023505575, 'prompt_tokens_details': None, 'queue_time': 0.086428005, 'total_time': 0.042567907}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd222-ab93-79e2-8244-3a7370d45082-0', usage_metadata={'input_tokens': 266, 'output_tokens': 10, 'total_tokens': 276})